In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
data=pd.read_excel("supplementary-data.xlsx")

C:\Users\gaura\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
data.drop(columns=['Sample'],axis=1,inplace=True)

In [4]:
data

,Water fraction [0-1],Rotational speed (rpm),Compression ratio,Ignition Advance Angle (°BTDC),Fuel temperature (℃),Exhaust temperature (℃),Oil temperature (℃),Torque (N m),Fuel consumption (g/s),Power (kW),Specific fuel consumption (g/kWh),Efficiency (%),CO (%),CO2 (%),O2 (%),HC (ppm),NOx (ppm)
0,0.0,2000,7.44,25.0,32.2,566.9,144.7,10.1,0.22,2.115339,374.408064,35.744154,0.93,7.4,14.9,56,483
1,0.0,2000,7.44,25.0,32.0,568.6,142.5,10.0,0.23,2.094395,395.340879,33.851545,0.95,7.5,14.8,57,483
2,0.0,2000,7.44,25.0,31.9,566.5,140.8,10.1,0.22,2.115339,374.408064,35.744154,0.96,7.4,14.8,59,480
3,0.0,2000,7.44,25.0,31.3,572.3,142.1,10.1,0.23,2.115339,391.426613,34.190061,0.89,7.8,14.3,46,575
4,0.0,2000,7.44,25.0,31.2,572.7,141.6,10.1,0.23,2.115339,391.426613,34.190061,0.85,7.7,14.3,45,567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.5,4000,9.44,63.0,30.4,661.1,70.2,9.3,1.16,3.895575,1071.985552,36.688030,0.68,11.9,12.5,1122,0
356,0.5,4000,9.44,63.0,30.4,661.5,69.7,9.8,1.13,4.105014,990.983125,39.686890,0.64,12.1,12.4,1010,0
357,0.5,4000,9.44,63.0,30.5,661.6,68.1,9.1,1.02,3.811799,963.324645,40.826359,0.68,12.0,12.4,1229,0
358,0.5,4000,9.44,63.0,30.7,659.4,69.5,9.3,1.02,3.895575,942.607986,41.723642,0.72,12.1,12.2,1116,0


In [10]:
#### from sklearn.preprocessing import MinMaxScaler

grouping_cols = ["Water fraction [0-1]", "Rotational speed (rpm)", "Compression ratio", "Ignition Advance Angle (°BTDC)"]

avg_values = data.groupby(grouping_cols).agg({
    "Efficiency (%)": "mean",
    "HC (ppm)": "mean",
    "NOx (ppm)": "mean"
}).reset_index()

scaler = MinMaxScaler()
avg_values[["Efficiency (%)", "HC (ppm)", "NOx (ppm)"]] = scaler.fit_transform(avg_values[["Efficiency (%)", "HC (ppm)", "NOx (ppm)"]])


alpha, beta, gamma =0,1,1



avg_values["Score"] = alpha * avg_values["Efficiency (%)"] - beta * avg_values["HC (ppm)"] - gamma * avg_values["NOx (ppm)"]


best_combination = avg_values.loc[avg_values["Score"].idxmax()]


optimized_params = best_combination[grouping_cols]
efficiency = best_combination["Efficiency (%)"]
hc = best_combination["HC (ppm)"]
nox = best_combination["NOx (ppm)"]


original_values = data.groupby(grouping_cols).agg({
    "Efficiency (%)": "mean",
    "HC (ppm)": "mean",
    "NOx (ppm)": "mean"
}).reset_index()

best_original_values = original_values[
    (original_values["Water fraction [0-1]"] == best_combination["Water fraction [0-1]"]) &
    (original_values["Rotational speed (rpm)"] == best_combination["Rotational speed (rpm)"]) &
    (original_values["Compression ratio"] == best_combination["Compression ratio"]) &
    (original_values["Ignition Advance Angle (°BTDC)"] == best_combination["Ignition Advance Angle (°BTDC)"])
]


if not best_original_values.empty:
    efficiency = best_original_values["Efficiency (%)"].values[0]
    hc = best_original_values["HC (ppm)"].values[0]
    nox = best_original_values["NOx (ppm)"].values[0]

# Display results
print("\n🔹 Best Optimized Parameters🔹")
print(optimized_params)
print(f"\n Efficiency (%): {efficiency:.4f}")
print(f" HC (ppm): {hc:.4f}")
print(f" NOx (ppm): {nox:.4f}")



🔹 Best Optimized Parameters🔹
Water fraction [0-1]                 0.20
Rotational speed (rpm)            2000.00
Compression ratio                    7.44
Ignition Advance Angle (°BTDC)      20.00
Name: 20, dtype: float64

 Efficiency (%): 38.0010
 HC (ppm): 23.8333
 NOx (ppm): 123.6667
